In [1]:
import os
import time
import copy
import argparse
from unittest import TestLoader
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image
from utils import data_preparation, gen_data_partition_iid, get_network, get_eval_pool,  get_time, make_client_dataset_from_partition, ParamDiffAug 
from utils import get_loops, get_dataset, gen_data_partition_dirichlet, evaluate_synset, get_daparam, match_loss, TensorDataset, epoch, DiffAugment

import random
from client import ClientDC
from server import ServerDC

In [2]:
def argparser():
    parser = argparse.ArgumentParser(description='Federated DC using gradient matching')

    # default args - data set and model
    parser.add_argument('--dataset', type=str, default='CIFAR10', help='dataset')
    parser.add_argument('--data_path', type=str, default='data', help='dataset path')
    parser.add_argument('--model', type=str, default='ConvNet', help='model')

    # default args - condensation
    parser.add_argument('--method', type=str, default='DC', help='DC/DSA')
    parser.add_argument('--dsa_strategy', type=str, default='None', help='differentiable Siamese augmentation strategy')  
    parser.add_argument('--init', type=str, default='noise', help='noise/real: initialize synthetic images from random noise or randomly sampled real images.')
    parser.add_argument('--ipc', type=int, default=1, help='image(s) per class')
    parser.add_argument('--dis_metric', type=str, default='ours', help='distance metric for gradient matching')
    parser.add_argument('--lr_img', type=float, default=0.1, help='learning rate for updating synthetic images')
    parser.add_argument('--lr_net', type=float, default=0.01, help='learning rate for updating network parameters')

    # default args - evluation
    parser.add_argument('--eval_mode', type=str, default='S', help='eval_mode') # S: the same to training model, M: multi architectures,  W: net width, D: net depth, A: activation function, P: pooling layer, N: normalization layer,
    parser.add_argument('--num_eval', type=int, default=20, help='the number randomly initialized models for evaluating the syn data')
    parser.add_argument('--epoch_eval_train', type=int, default=300, help='epochs to train a model with synthetic data for evaluating the syn data')

    # default args - experiments
    parser.add_argument('--num_exp', type=int, default=5, help='the number of experiment runs')  
    parser.add_argument('--Iteration', type=int, default=1000, help='the number of model intializations to be used, i.e., big K in the paper gradient matching')

    # args - fed/overall
    parser.add_argument('--seed', type=int, default=3, help='set a seed for reproducability, set to 0 to activate randomness')
    parser.add_argument('--num_clients', type=int, default=5, help='number of clients')
    parser.add_argument('--client_alpha', type=float, default=100.0, help='dirichlet alpha that controls the non-iid degree')
    
    # args - fed/server
    parser.add_argument('--stand_alone', action='store_true', default=False, help='trigger non-federated local training mode')
    parser.add_argument('--server_mode', type=str, default='train', help='operation model of server train or agg')
    parser.add_argument('--server_lr', type=float, default=0.01, help='learning rate for updating global model by the server')
    parser.add_argument('--server_batch_train', type=int, default=128, help='batch size for training networks')
    parser.add_argument('--server_epoch_train', type=int, default=10, help='epochs to train the global model with synthetic data')
    
    # args - fed/clients
    parser.add_argument('--rounds', type=int, default=10, help='epochs to train the global model with synthetic data')
    parser.add_argument('--client_epoch_train', type=int, default=10, help='epochs to train the local model with synthetic data')
    parser.add_argument('--client_batch_train_data', type=int, default=256, help='batch size for real data')
    parser.add_argument('--client_batch_train_model', type=int, default=128, help='batch size for training networks')

    # args - results
    parser.add_argument('--save_results', action='store_true', default=False, help='use this to save trained synthetic data and images')
    parser.add_argument('--save_root', type=str, default='result', help='path to save results')
    args = parser.parse_args()

    if args.stand_alone:
        save_tag = args.dataset + '_local' + time.strftime('_%y-%m-%d-%H-%M-%S') 
    elif args.server_mode == 'agg': # use model aggregation mode
        save_tag = args.dataset + '_fed_agg' + time.strftime('_%y-%m-%d-%H-%M-%S') 
    elif args.server_mode == 'train': # use model training mode
        save_tag = args.dataset + '_fed_train' + time.strftime('_%y-%m-%d-%H-%M-%S') 
    args.save_path = os.path.join(args.save_root, save_tag) 

    # the following are original arguments better to be kept unchanged
    # args.outer_loop, args.inner_loop = 10, 10
    # args.outer_loop, args.inner_loop = get_loops(args.ipc)
    args.dsa_param = ParamDiffAug()
    args.dsa = False if args.dsa_strategy in ['none', 'None'] else True
    args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    return args

In [3]:
import sys
sys.argv=['']
del sys

args=argparser()

In [4]:
args.dataset = 'MNIST'
args.save_results = True
args.ipc = 2
args.num_exp = 1
args.num_eval = 1
args.epoch_eval_train = 1
args.Iteration=10
print(args)

Namespace(method='DC', dataset='MNIST', model='ConvNet', ipc=2, eval_mode='S', num_exp=1, num_eval=1, epoch_eval_train=1, Iteration=10, lr_img=0.1, lr_net=0.01, batch_real=256, batch_train=256, init='noise', dsa_strategy='None', data_path='data', save_root='result', dis_metric='ours', num_clients=5, seed=3, client_alpha=100.0, stand_alone=False, save_results=True, outer_loop=10, inner_loop=10, dsa_param=<utils.ParamDiffAug object at 0x0000020D6693A280>, dsa=False, save_path='result\\CIFAR10_fed_22-08-19-11-51-57_notebook', device='cuda')


In [5]:
''' set seeds '''
if args.seed:
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
rng = np.random.default_rng(args.seed)    

''' some global setup '''
# if not os.path.exists(args.data_path):
#     os.mkdir(args.data_path)
if not os.path.exists(args.save_path):
    os.mkdir(args.save_path)

# The list of iterations when we evaluate models and record results.
# the defualt setting is to evaluate every 500 iterations, i.e., k=n*500
# eval_it_pool = np.arange(0, args.Iteration+1, 500).tolist() if args.eval_mode == 'S' or args.eval_mode == 'SS' else [args.Iteration] 
eval_it_pool = [args.Iteration] # only evaluate at last iteration
print('eval_it_pool: ', eval_it_pool)
model_eval_pool = get_eval_pool(args.eval_mode, args.model, args.model)
data_set, data_info, testloader_server = data_preparation(args.dataset)

accs_all_clients_all_exps = [dict() for i in range(args.num_clients)]
for i in range(args.num_clients):
    for key in model_eval_pool:
        accs_all_clients_all_exps[i][key]=[]
data_save_all_clients = [[] for i in range(args.num_clients)]

eval_it_pool:  [10]


In [6]:
# looping over multiple experiment trials
for exp in range(args.num_exp):
    print('\n================== Exp %d ==================\n '%exp)
    print('Hyper-parameters: \n', args.__dict__)
    print('Evaluation model pool: ', model_eval_pool)

    # split data set for each client'''       
    # generate training data partitioning using IID method
    client_train_data_idcs, client_train_class_dict = gen_data_partition_iid(
        data=data_set['train_data'], 
        num_classes=data_info['num_classes'], 
        labels=data_set['train_labels'], 
        data_mapp=data_set['mapp'],
        num_clients=args.num_clients, 
        generator=rng, 
        verbose_hist=False)

    client_test_data_idcs, client_test_class_dict = gen_data_partition_iid(
        data=data_set['test_data'], 
        num_classes=data_info['num_classes'], 
        labels=data_set['test_labels'], 
        data_mapp=data_set['mapp'],
        num_clients=args.num_clients, 
        generator=rng, 
        verbose_hist=False)

    # make client data using the generated partition
    client_data_train = make_client_dataset_from_partition(data_set['train_data'], args.num_clients, client_train_data_idcs)
    client_data_test = make_client_dataset_from_partition(data_set['test_data'], args.num_clients, client_test_data_idcs)

    # set the architecture for the network to be trained
    net_train = get_network(args.model, data_info['channel'], data_info['num_classes'], data_info['img_size']).to(args.device)

    # create clients and server
    clients = [ClientDC(id, args, net_train, data_info, client_data_train[i], client_data_test[i], eval_it_pool, model_eval_pool) for id in range(args.num_clients)]
    for client in clients:
        print('Client {} has {} training samples {} testing samples'.format(client.id, client.num_local_data_train, client.num_local_data_test))
    server = ServerDC(args, net_train, clients, data_info)
    print('FL server created.')

    # organize the real dataset and initialize the synthetic data
    for client in clients:
        client.organize_local_real_data()
        for ch in range(client.channel):
            print('real images channel %d, mean = %.4f, std = %.4f'%(ch, torch.mean(client.images_all[:, ch]), torch.std(client.images_all[:, ch])))
        client.syn_data_init()
        client.data_trainer_setup() # only to be called after syn_data_init()

    # training starts from here
    print('%s training begins'%get_time())
    
    # NOTE this loop is over the different model initializations, i.e., the loop indixed by K in the paper, Algorithm 1 line 4
    for it in range(args.Iteration+1): 
        # get a new random initialization of the network
        server.global_model = get_network(args.model, data_info['channel'], data_info['num_classes'], data_info['img_size']).to(args.device) 
        server.global_model_state = server.global_model.state_dict()
        
        for client in clients:
            # # get a new random initialization of the network
            # client.model_train = get_network(args.model, client.channel, client.num_classes, client.im_size).to(args.device) 

            # fetch newly intialized server model weights
            client.sync_with_server(server, method='state')

            # set the optimizer for learning synthetic data
            optimizer_net = client.net_trainer_setup(client.model_train)


        # NOTE this loop is indixed by T in the paper, Algorithm 1 line 4
        # this loop resembles the communication round in FL
        for ol in range(args.rounds): 

            # clients perform local update of data and network '''
            for client in clients:
                if not args.stand_alone and ol:
                    client.sync_with_server(server, method='state') # fetch server model weights

                # freeze the running mu and sigma for BatchNorm layers '''
                # Synthetic data batch, e.g. only 1 image/batch, is too small to obtain stable mu and sigma.
                # So, we calculate and freeze mu and sigma for BatchNorm layer with real data batch ahead.
                # This would make the training with BatchNorm layers easier.
                BN_flag = False
                BNSizePC = 16  # for batch normalization
                for module in client.model_train.modules():
                    if 'BatchNorm' in module._get_name(): #BatchNorm
                        BN_flag = True
                if BN_flag:
                    img_real = torch.cat([client.get_images(c, BNSizePC) for c in range(client.num_classes)], dim=0)
                    client.model_train.train() # for updating the mu, sigma of BatchNorm
                    output_real = client.model_train(img_real) # get running mu, sigma
                    for module in client.model_train.modules():
                        if 'BatchNorm' in module._get_name():  #BatchNorm
                            module.eval() # fix mu and sigma of every BatchNorm layer

                # local update of synthetic data '''
                # one step of SGD, can be repeated for multiple steps
                # update only once but over T iterations equivalent to T steps of SGD for learning the data
                client.syn_data_update(client.model_train) 

                # local update of network (using synthetic data) '''
                client.network_update(client.model_train, optimizer_net) 
                client.local_model_state = copy.deepcopy(client.model_train.state_dict()) # copy the updated local model weights to another iterables to avoid any unaware modification   

            # Server perform model aggregation upon local network updates
            if not args.stand_alone:
                server.net_weights_aggregation(clients)

        # Evaluate synthetic data trained in last iteration
        for client in clients:
            client.syn_data_eval(exp, it, accs_all_clients_all_exps)
            client.loss_avg /= (client.num_classes*args.rounds) # Summary for client data condensation for this exp trial

            if not os.path.exists(client.save_path):
                os.mkdir(client.save_path)
            
            if it%10 == 0:
                print('%s iter = %04d, loss = %.4f' % (get_time(), it, client.loss_avg))
            if it == args.Iteration and args.save_results: # only record the final results
                data_save_all_clients[client.id].append([copy.deepcopy(client.image_syn.detach().cpu()), copy.deepcopy(client.label_syn.detach().cpu())])
                torch.save({'data': data_save_all_clients[client.id], 'accs_all_exps': accs_all_clients_all_exps[client.id], }, os.path.join(client.save_path, 'res_%s_%s_%s_%dipc.pt'%(client.args.method, client.args.dataset, client.args.model, client.args.ipc)))





================== Exp 0 ==================
 
Hyper-parameters: 
 {'method': 'DC', 'dataset': 'MNIST', 'model': 'ConvNet', 'ipc': 2, 'eval_mode': 'S', 'num_exp': 1, 'num_eval': 1, 'epoch_eval_train': 1, 'Iteration': 10, 'lr_img': 0.1, 'lr_net': 0.01, 'batch_real': 256, 'batch_train': 256, 'init': 'noise', 'dsa_strategy': 'None', 'data_path': 'data', 'save_root': 'result', 'dis_metric': 'ours', 'num_clients': 5, 'seed': 3, 'client_alpha': 100.0, 'stand_alone': False, 'save_results': True, 'outer_loop': 10, 'inner_loop': 10, 'dsa_param': <utils.ParamDiffAug object at 0x0000020D6693A280>, 'dsa': False, 'save_path': 'result\\CIFAR10_fed_22-08-19-11-51-57_notebook', 'device': 'cuda'}
Evaluation model pool:  ['ConvNet']
Client 0 has 12000 training samples 2000 testing samples
Client 1 has 12000 training samples 2000 testing samples
Client 2 has 12000 training samples 2000 testing samples
Client 3 has 12000 training samples 2000 testing samples
Client 4 has 12000 training samples 2000 testin

c:\Users\YWANG\atr-data-distillation\DatasetCondensation\client.py:83: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  self.label_syn = torch.tensor([np.ones(self.ipc)*i for i in range(self.num_classes)], dtype=torch.long, requires_grad=False, device=self.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]


class c = 0: 1163 real images
class c = 1: 1346 real images
class c = 2: 1175 real images
class c = 3: 1237 real images
class c = 4: 1168 real images
class c = 5: 1122 real images
class c = 6: 1178 real images
class c = 7: 1274 real images
class c = 8: 1157 real images
class c = 9: 1180 real images
real images channel 0, mean = 0.0005, std = 1.0009
initialize synthetic data from random noise
class c = 0: 1163 real images
class c = 1: 1346 real images
class c = 2: 1175 real images
class c = 3: 1237 real images
class c = 4: 1168 real images
class c = 5: 1122 real images
class c = 6: 1178 real images
class c = 7: 1274 real images
class c = 8: 1157 real images
class c = 9: 1180 real images
real images channel 0, mean = 0.0005, std = 1.0009
initialize synthetic data from random noise
class c = 0: 1163 real images
class c = 1: 1346 real images
class c = 2: 1175 real images
class c = 3: 1237 real images
class c = 4: 1168 real images
class c = 5: 1122 real images
class c = 6: 1178 real images


In [7]:
print('\n==================== Final Results ====================\n')
for key in model_eval_pool:
    acc_overall = []
    for i in range(args.num_clients):
        accs = accs_all_clients_all_exps[i][key]
        print('Client %d run %d experiments, train on %s, evaluate %d random %s, mean  = %.2f%%  std = %.2f%%'%(i, args.num_exp, args.model, len(accs), key, np.mean(accs)*100, np.std(accs)*100))
        acc_overall += accs
    print('\n-------------------------\nAverage performance after {:d} experiments, evaluted on {:2d} random {}, mean = {:.2f}%, std = {:.2f}%'.format(args.num_exp, len(acc_overall), args.model, np.mean(acc_overall)*100, np.std(accs)*100))



==================== Final Results ====================

Client 0 run 1 experiments, train on ConvNet, evaluate 1 random ConvNet, mean  = 89.65%  std = 0.00%
Client 1 run 1 experiments, train on ConvNet, evaluate 1 random ConvNet, mean  = 83.20%  std = 0.00%
Client 2 run 1 experiments, train on ConvNet, evaluate 1 random ConvNet, mean  = 89.15%  std = 0.00%
Client 3 run 1 experiments, train on ConvNet, evaluate 1 random ConvNet, mean  = 87.20%  std = 0.00%
Client 4 run 1 experiments, train on ConvNet, evaluate 1 random ConvNet, mean  = 87.85%  std = 0.00%

-------------------------
Average performance after 1 experiments, evaluted on  5 random ConvNet, mean = 87.41%, std = 0.00%


The following is for debug

In [8]:
the_client = clients[0]

In [11]:
the_client.label_syn

tensor([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9],
       device='cuda:0')

In [12]:
another_client = clients[1]

In [20]:
all_data = []
all_data.append(the_client.image_syn)
all_data.append(another_client.image_syn)

In [23]:
torch.tensor(all_data)

ValueError: only one element tensors can be converted to Python scalars

In [28]:
the_client.image_syn[0].size()

torch.Size([1, 28, 28])

In [31]:
all_data = torch.cat((the_client.image_syn, another_client.image_syn))

In [32]:
all_data.size()

torch.Size([40, 1, 28, 28])

In [34]:
all_label = torch.cat((the_client.label_syn,another_client.label_syn))

In [35]:
all_label

tensor([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 0, 0, 1, 1,
        2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9], device='cuda:0')

In [36]:
image_syn_train, label_syn_train = copy.deepcopy(all_data.detach()), copy.deepcopy(all_label.detach()) 

In [37]:
dataset_syn_train = TensorDataset(image_syn_train, label_syn_train)
net_trainloader = torch.utils.data.DataLoader(dataset_syn_train, batch_size=64, shuffle=True, num_workers=0)

In [38]:
dataset_syn_train

In [40]:
dataset_syn_train.images.size()

torch.Size([40, 1, 28, 28])

In [92]:
all_label = clients[0].label_syn
for id in range(1,args.num_clients):
    all_label = torch.cat((all_label, clients[id].label_syn))

In [93]:
len(all_label)

100

In [97]:
all_data = copy.deepcopy(clients[0].image_syn)
for id in range(1,args.num_clients):
    all_data = torch.cat((all_data, copy.deepcopy(clients[id].image_syn)))

In [98]:
len(all_data)

100

In [99]:
image_syn_train, label_syn_train = copy.deepcopy(all_data.detach()), copy.deepcopy(all_label.detach()) 
dataset_syn_train = TensorDataset(image_syn_train, label_syn_train)
net_trainloader = torch.utils.data.DataLoader(dataset_syn_train, batch_size=64, shuffle=True, num_workers=0)

In [101]:
len(dataset_syn_train)

100

In [90]:
del all_data, all_label, image_syn_train, label_syn_train, dataset_syn_train, net_trainloader